In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
!pip install ratsnlp

In [1]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    'skt/kogpt2-base-v2',
)
model.eval()

ModuleNotFoundError: ignored

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    'skt/kogpt2-base-v2',
    eos_token='</s>'
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


프롬프트 준비

In [ ]:
input_ids = tokenizer.encode('안녕하세요', return_tensors='pt')
input_ids

tensor([[25906,  8702,  7801,  8084]])

- 그리디 서치(greedy search)는 매순간 최선(best)를 선택해 탐색 범위를 줄여보자는 것이 핵심 아이디어
- 빔 서치(beam search)는 빔(beam) 크기만큼의 선택지를 계산 범위에 포함
- 탑k 샘플링(top-k sampling)은 
  - 모델이 예측한 다음 토큰 확률 분포 에서 확률값이 가장 높은  k 개 토큰 가운데 하나를 다음 토큰으로 선택하는 기법. 
  - k개 안에 있는 단어라면 의자 같이 확률값이 낮은 케이스도 다음 토큰으로 추출될 수 있습니다. 따라서 탑k 샘플링은 매 시행 때마다 생성 결과가 달라집니다.
  - top_k를 1로 입력한다면 do_sample 인자를 True로 두더라도 그리디 서치와 동일한 효과
- 템퍼러처 스케일링(temperature scaling)이란 
  - 모델의 다음 토큰 확률분포에 변형을 가해 문장을 다양하게 생성하는 기법
  - 확률분포를 변형한다는 의미는, 대소 관계의 역전 없이 분포의 모양만을 바꾼다는 의미
  - 이 값이 0에 가까울 수록 확률분포 모양이 원래 대비 뾰족해 진다. 순위의 변동은 없지만 원래 컸던 확률은 더 커지고, 작았던 확률은 더 작아져 확률분포의 모양이 뾰족(sharp)해진다. 그만큼 확률값 기준 1등 토큰이 다음 토큰으로 뽑힐 가능성이 높아진다
  - temperature를 1보다 작게 하면 상대적으로 정확한 문장을, 1보다 크게 하면 상대적으로 다양한 문장을 생성한다.
  - 템퍼러처 스케일링은 탑k 샘플링, 탑p 샘플링과 같이 적용해야 의미가 있다.
- 탑p 샘플링(top-p sampling)은 
  - 확률값이 높은 순서대로 내림차순 정렬을 한 뒤 누적 확률값이  p 이하인 단어들 가운데 하나를 다음 단어로 선택하는 기법
  - 뉴클리어스 샘플링(necleus sampling)이라고도 불리며 확률값을 기준으로 단어들을 내림차순 정렬해 그 값이 높은 단어들을 후보로 삼는다는 점에서는 탑k 샘플링과 같지만 상위  k개를 후보로 삼느냐(탑k 샘플링), 누적 확률값이  p
  이하인 단어들을 후보로 삼느냐(탑p 샘플링)에 따라 차이
- 리피티션 패널티(repetition penalty)라는 방식으로 반복을 통제할 수도 있습니다. repetition_penalty라는 인자를 주면 됩니다. 그 값은 1.0 이상이어야 하며 클 수록 페널티가 세게 적용

Greedy Search

다음 단어 확률 분포에서 최대 확률을 내는 단어들을 리턴합니다. 
여러 번 수행하더라도 생성 결과가 바뀌지 않습니다 (`do_sample=False`).
`max_length`는 생성 최대 길이이며 이보다 길거나, 짧더라도 EOD 토큰 등 스페셜 토큰이 나타나면 생성을 중단합니다. `min_length`는 생성 최소 길이이며 이보다 짧은 구간에서 EOD 등 스페셜 토큰이 등장해 생성이 중단될 경우 해당 토큰이 나올 확률을 0으로 수정하여 문장 생성이 종료되지 않도록 강제합니다

In [ ]:
import torch
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50
  )

In [ ]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



Beam Search 

Beam Search는 다음 단어 확률 분포에서 `num_beams`만큼의 경우의 수를 남겨가면서 문장을 생성합니다. Beam search는 Greedy search보다 계산량이 많지만 좀 더 확률값이 높은 문장을 생성할 수 있습니다.

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
      num_beams=3
  )

In [ ]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그렇지 않습니다."
"그


반복 줄이기

반복되는 n-gram 사이즈를 지정하기

위의 예시를 보면 `"그럼, 그건 뭐예요?"`이 반복됩니다. 이를 아래와 같이 지정해 반복을 방지합니다. 3개 이상의 토큰이 반복될 경우 해당 3-gram 등장 확률을 0으로 만들어 생성 결과에서 배제합니다.

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
      no_repeat_ngram_size=5 #토큰 3개 이상 반보시 3번째 토큰 확률 0으로 변경
  )

In [ ]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"아니요, 그건 뭐예요."
"그럼요, 그건 뭐죠?"
"그건 뭐예요!"
"그건 무슨 말씀


repetition penalty

repetition penalty로 반복을 통제할 수도 있습니다. 다음과 같이 실행하면 되며 그 범위는 1 이상의 값을 가져야 합니다. 1이라면 아무런 패널티를 적용하지 않는게 됩니다.

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=False,
      min_length=10,
      max_length=50,
      repetition_penalty = 1.5
  )

In [ ]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요, 아저씨. 저는 지금 이 순간에도 괜찮아요. 그리고 제가 할 수 있는 일은 아무것도 없어요.
이제 그만 돌아가고 싶어요.
제가 하는 일이 무엇


top-k sampling

지금까지는 생성을 반복하더라도 그 결과가 동일한 샘플링 방식을 살펴봤습니다. top-k sampling은 다음 단어를 뽑을 때 확률값 기준 가장 큰 k개 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, k개 안에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-k sampling은 매 시행 때마다 생성 결과가 달라집니다. k는 1 이상의 값을 지녀야 합니다.

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample=True,
      min_length=10,
      max_length=50,
      top_k = 50 #확률값이 가장 높은 k개 토큰 가운데 하나를 다음 토큰으로 선택
  )

In [ ]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요'를, "가족을 위해 사랑을 실천하는 것도 중요하지만 그걸 지켜 나가는 건 나 하나만큼은 해야 한다"는 등의 내용이 있다.
실제 한 누리꾼은 "결혼을 준비 중인 부부가 신혼여행을 가든 다른 사람이 결혼


top-k sampling + temperature scaling

top-k sampling은 temperature scaling과 동시에 적용할 수 있습니다. 그 값에 따라 다음과 같은 효과가 납니다.

t가 0에 가까워질 수록 토큰 분포가 sharp해진다 > 1등 토큰이 뽑힐 확률이 그만큼 높아진다 > do_sample=True이지만 사실상 greedy decoding이 된다

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = True,
      min_length = 10,
      max_length = 50,
      top_k = 50,
      temperature = 0.01
  )

In [ ]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"
"그럼, 그건 뭐예요?"



top-p sampling

top-p sampling은 다음 단어를 뽑을 때 누적 확률값이 p 이하인 단어들 가운데 하나를 선택하는 기법입니다. 확률값이 큰 단어가 다음 단어로 뽑힐 가능성이 높아지지만, 누적 확률값 p 이하에 있는 단어라면 확률값이 낮더라도 다음 단어로 추출될 수 있습니다. 따라서 top-p sampling은 매 시행 때마다 생성 결과가 달라집니다. p는 확률이기 때문에 0~1 사이의 값을 지녀야 합니다. p가 1이라면 어휘 집합에 있는 모든 단어를 대상으로 샘플링하기 때문에 top-p sampling 효과가 사라집니다. p가 1보다 약간 작다면 확률값이 낮은 일부 단어들을 다음 단어 후보에서 제거해 생성 품질을 높입니다

In [ ]:
with torch.no_grad():
  generated_ids = model.generate(
      input_ids,
      do_sample = True,
      min_length = 10,
      max_length = 50,
      top_p = 0.92,
  )

In [ ]:
print(tokenizer.decode([el.item() for el in generated_ids[0]]))

안녕하세요”라고 인사를 건넸고 그는 또 “박근혜 대통령 취임 당시 한진해운을 살리겠다고 했다”고 강조했다.
당시 박 대통령은 한진해운과 아시아나가 합쳐지면 한국 경제의 성장엔진은 사라질 것이라며 “정부가
